In [ ]:
# Demonstration of leveraging Azure OpenAI to improve accuracy of results through re-ranking 
# of results retrieved from an Azure AI Search service

# IMPORTANT: Update config.json with your Azure OpenAI Service details

In [ ]:
import os 
import json
import time
from datetime import datetime
from openai import AzureOpenAI
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt 
from scipy.spatial.distance import cosine  


In [8]:
openai_temperature = 0.1

#Load the configuration details for the Azure OpenAI Instance
#Credentials should be secured using a more secure method such as Azure KeyVault
config = json.load(open("config.json"))

#Azure OpenAI
openai_embedding_api_base = config["openai_embedding_api_base"]
openai_embedding_api_key = config["openai_embedding_api_key"]
openai_embedding_api_version = config["openai_embedding_api_version"]
openai_embeddings_model = config["openai_embedding_model"]

openai_gpt_api_base = config["openai_gpt_api_base"]
openai_gpt_api_key = config["openai_gpt_api_key"]
openai_gpt_api_version = config["openai_gpt_api_version"]
openai_gpt_model = config["openai_gpt_model"]

# gets the API Key from environment variable AZURE_OPENAI_API_KEY
embeddings_client = AzureOpenAI(
    api_version=openai_embedding_api_version,
    azure_endpoint=openai_embedding_api_base,
    api_key=openai_embedding_api_key
)

gpt_client = AzureOpenAI(
    api_version=openai_gpt_api_version,
    azure_endpoint=openai_gpt_api_base,
    api_key=openai_gpt_api_key
)

print ('Azure OpenAI Embeddings Base URL:', openai_embedding_api_base)
print ('Azure OpenAI Embeddings Model:', openai_embeddings_model)
print ('Azure OpenAI GPT Base URL:', openai_gpt_api_base)
print ('Azure OpenAI GPT Model:', openai_gpt_model)


Azure OpenAI Embeddings Base URL: https://YOUR_SERVICE.openai.azure.com/
Azure OpenAI Embeddings Model: text-embedding-3-large
Azure OpenAI GPT Base URL: https://YOUR_SERVICE.openai.azure.com/
Azure OpenAI GPT Model: gpt-4


In [3]:
# Function to generate vectors content
max_attempts = 6
max_backoff = 60
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(max_attempts))
def generate_embedding(text):
    try:
        # text-embedding-3-small == 1536 dims
        response = embeddings_client.embeddings.create(
            input=text,
            model=openai_embeddings_model
        )
        return json.loads(response.model_dump_json())["data"][0]['embedding']
    except Exception as ex:
        print ('Error - Retry count:', ex)
    return None

# Determine the relevancy of text based on a question and return confidence as well as relevent text from text
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def rerank_documents(question, data):
    system_prompt = """
        I am going to supply you with a set of potential answers and your goal is to determine which of them is best able to answer the question: \n""" +  question + """
        Please respond in JSON format with a "confidence" score for each example indicating your confidence the text answers the question as well as the "id" of the text.  
        Please also include a field called "relevent_text" which includes the text that is relevent to being able to answer the question.  

        Each example will include an answer id as well as the text for the potential answer, separated by a colon.  
    """

    user_prompt = ""
    for d in data:
        user_prompt += d[0] + ": " + d[1] + "\n"

    try:
        response = gpt_client.chat.completions.create(
            model=openai_gpt_model, 
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            response_format={ "type": "json_object" },
            temperature=openai_temperature,
            max_tokens=4096,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None
        )
        json_data = json.loads(response.choices[0].message.content)
        return json_data
    except Exception as ex:
        print ('Error - Retry count:', ex)
    
    return {}

        


In [4]:
# Determine the Cosine Similarity of the query and answers (to understand semantics vs intent)
question_emb = generate_embedding('Does the applicant have any significant illnesses in his medical history?')

answer_1_emb = generate_embedding('Please use application form 354-01 to enter applicants medical history, significant illnesses and other symptoms.')
answer_2_emb = generate_embedding('Mr. John Doe, a 35-year-old non-smoker, is applying for a life insurance policy. He works as an accountant and leads a low-risk lifestyle. He exercises regularly and maintains a healthy diet. His medical history reveals no significant illnesses, and his family history is also clear of any hereditary diseases. He is interested in a policy with a coverage amount of $500,000')

print("Cosine Similarity of Question to Answer 1:", 1 - cosine(question_emb, answer_1_emb))  
print("Cosine Similarity of Question to Answer 2:", 1 - cosine(question_emb, answer_2_emb))  



Cosine Similarity of Question to Answer 1: 0.5595185612023936
Cosine Similarity of Question to Answer 2: 0.39874486454438407


In [5]:
# Score the answers based on intent and retrieve relevent text
question = "Does the applicant have any significant illnesses in his medical history?"

data = [
    ["1", "Please use application form 354-02 to enter applicants medical history, significant illnesses and other symptoms."],
    ["2", "Mr. John Doe, a 35-year-old non-smoker, is applying for a life insurance policy. He works as an accountant and leads a low-risk lifestyle. He exercises regularly and maintains a healthy diet. His medical history reveals no significant illnesses, and his family history is also clear of any hereditary diseases. He is interested in a policy with a coverage amount of $500,000"],
]
print(json.dumps(rerank_documents(question, data), indent=4))

{
    "answers": [
        {
            "id": 1,
            "confidence": 0.1,
            "relevent_text": "Please use application form 354-02 to enter applicants medical history, significant illnesses and other symptoms."
        },
        {
            "id": 2,
            "confidence": 0.9,
            "relevent_text": "His medical history reveals no significant illnesses, and his family history is also clear of any hereditary diseases."
        }
    ]
}
